In [55]:
import numpy as np
import pandas as pd


In [86]:
# Functions from "dot" to "pnt2line" were inspired in the 
# "Shortest Distance from a Point to a Line" project: 
# http://www.fundza.com/vectors/point2line/index.html

def pnt2line(pnt, start, end):
    """ 
    Calcula la distancia mínima entre un punto y una recta

    Esta distancia mínima puede ser:
        - La distancia ortogonal entre el punto y la recta 
        - La distancia entre el punto y el inicio de la recta
        - La distancia entre el punto y el final de la recta

    Parameters
    ----------
    pnt: np.array
        coordenadas en el plano cartesiano del punto
    start: np.array
        coordenadas en el plano cartesiano del inicio de la recta
    end: np.array
        coordenadas en el plano cartesiano del final de la recta

    Returns
    -------
    dist: Valor escalar
        Distancia mínima calculada entre el punto y la recta
    nearest: np.array
        coordenadas sobre la recta, desde dónde la distancia es
        mínima hacia el punto
    """
    line_vec = end-start # se genera vector
    pnt_vec = pnt-start # se genera vector
    line_len = np.linalg.norm(line_vec) # magnitud del vector
    line_unitvec = line_vec/np.linalg.norm(line_vec) # vector unitario
    pnt_vec_scaled = pnt_vec*(1.0/line_len) # Escalamiento
    t = np.dot(line_unitvec, pnt_vec_scaled)
    if t < 0.0:
        t = 0.0
    elif t > 1.0:
        t = 1.0
    nearest = line_vec*t # Escalamiento
    dist = np.linalg.norm(pnt_vec-nearest) # Distancia entre puntos
    nearest = nearest+start
    return dist, nearest


def haversine(coord1, coord2):
    """
    Cálculo de la distancia (en metros) entre 2 coordenadas de GPS.

    El resultado es una estimación determinada a través de la distancia 
    de círculo máximo sabiendo los puntos de latitud y longitud. Para este
    fin se hace uso de la ecuación "haversine".
    
    Parameters
    ----------
    coord1, coord2: list o tuple
        coordenadas (latitud, longitud) entre las cuales se desea conocer
        la distancia que las separa en metros.

    Returns
    -------
    Escalar
        Distancia en metros que separa ambos puntos
    """
    s_lat, s_lng = coord1
    e_lat, e_lng = coord2

    R = 6371000  # radius of the earth in meters

    s_lat = s_lat*np.pi/180.0
    s_lng = np.deg2rad(s_lng)
    e_lat = np.deg2rad(e_lat)
    e_lng = np.deg2rad(e_lng)

    d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat) * \
        np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2

    return 2 * R * np.arcsin(np.sqrt(d))


def get_axis(df, origin):
    """
    Se determinan los valores x,y dentro de un plano cartesiano para un
    dataframe que disponga de las coordenadas de latitud y longitud.

        Se toma cómo eje vertical, los valores de latitud
        Se toma cómo eje horizontal, los valores de longitud

    Parameters
    ----------
    df: pd.DataFrame
        objeto DataFrame que contiene en sus columnas las columnas:
            - lat: coordenadas de latitud
            - lon: cooordenadas de longitud
    origin:  pd.Series
        objeto Series que contiene las coordenadas de latitud (lat) y 
        longitud (lon) del origen del plano cartesiano.

    Returns
    -------
    pd.DataFrame
        Se retorna el mismo "df", con las columnas correspondientes con
        las coordenadas x,y al que corresponde cada coordenada GPS respecto
        del origen ingresado.
    """
    df["x_sign"] = (df["lon"]-origin.loc["lon"]) / \
        abs(df["lon"]-origin.loc["lon"])
    df["y_sign"] = (df["lat"]-origin.loc["lat"]) / \
        abs(df["lat"]-origin.loc["lat"])
    df["x_axis"] = df.apply(lambda x: haversine(
        origin, [origin[0], x["lon"]])*x[["x_sign"]], axis=1)
    df["y_axis"] = df.apply(lambda x: haversine(
        origin, [x["lat"], origin[1]])*x["y_sign"], axis=1)
    df = df.fillna(0).drop(columns=["x_sign", "y_sign"])
    return df


def axis_comp(route_axis, geofences_axis):
    """
    Construye una matriz que contiene la distancia que separan todos y
    cada uno de los puntos que componen los objetos ingresados.

    Esta función se pensó específicamente para comparar las coordenadas
    de los puntos de ruta sobre las filas, y las coordenadas de geocercas
    sobre las columnas

    Parameters
    ----------
    route_axis: pd.Series
        Coordenadas x o y de puntos de ruta
    geofences_axis: pd.Series
        Coordenadas x o y de geocercas

    Returns
    -------
    np.array
        matriz de dimensiones (# de puntos de ruta,# de geocercas) en dónde
        se encuentra la distancia que separan los puntos de ruta y geocercas
    """
    num_r = route_axis.values.shape[0]  # Number of points in route
    num_gf = geofences_axis.values.shape[0]  # Number of geofences
    # route points in the rows
    route_axis = np.reshape(route_axis.values, (num_r, 1))
    # geofences in the columns
    geofences_axis = np.reshape(geofences_axis.values, (1, num_gf))
    list_r = []
    for i in range(num_gf):
        list_r.append(route_axis)
    # route points concatenated horizontally by number of geofences
    route_axis = np.concatenate(list_r, axis=1)
    dist = route_axis-geofences_axis  # distance along x axis
    return dist


def euclidean_dist(df_route, df_geofences):
    """
    Calcula la distancia euclidiana entre los puntos de ruta y geocercas

    Parameters
    ----------
    df_route: pd.DataFrame
        dataframe de puntos de ruta con las coordenadas cartesianas
        debe tener las columnas:
            x_axis: coordenadas en x
            y_axis: coordenadas en y
    df_geofences: pd.DataFrame
        Coordenadas x o y de geocercas
        dataframe de puntos de geocerca con las coordenadas cartesianas
        debe tener las columnas:
            x_axis: coordenadas en x
            y_axis: coordenadas en y

    Returns
    -------
    pd.DataFrame
        DataFrame que contiene la distancia euclidiana entre los puntos de
        ruta y geocercas.
        index de filas: index de punto de ruta en el orden de df_route
        nombres de columnas: index de geocercas en el orden de df_geofences
    """
    dist_x = axis_comp(df_route.x_axis, df_geofences.x_axis) # distancias eje x
    dist_y = axis_comp(df_route.y_axis, df_geofences.y_axis) # distancias eje y
    dist = np.sqrt(np.square(dist_x)+np.square(dist_y))
    return pd.DataFrame(dist)


def gf_within_r(min_dist_df, df_route, df_geofences):
    """
    Se determina la ubicación de la geocerca entre los puntos de ruta apropiados

    Este orden se determina según la distancia entre la geocerca (punto) y 
    las rectas generadas entre el punto de distancia mínima y sus vecinos.

    Parameters
    ----------
    min_dist_df: pd.DataFrame
        DataFrame con la correspondencia entre id de geocercas y puntos de ruta
        en los cuales la distancia es mínima
    df_route: pd.DataFrame
        DataFrame con información asociada a los puntos de ruta
    df_geofences: pd.DataFrame
        DataFrame con información asociada a los puntos de geocerca

    Returns
    -------
    pd.DataFrame
        DataFrame con información relevante para el ordenamiento de las geocercas
        dentro de la ruta y la validación de la distancia mínima entre geocercas
        y ruta
        index: id de geocercas
        order: puntos de ruta entre los cuales la geocerca sería ubicada
        dist: distancia euclidiana mínima entre la geocerca y la recta generada
            entre los puntos de ruta indicados en la columna "order"
        gf_rad: radio de geocerca
        coord_gps_r: coordenadas gps de los puntos de ruta en "order"
        coord_gps_gf: coordenadas gps de los puntos de geocerca en el index
    """

    list_order = []
    list_dist = []
    list_gps_r = []
    list_rad_gf = []
    list_gps_gf = []
    for idx, row in min_dist_df.iterrows():
        # id de puntos de ruta y geocerca
        route_idx = row.r_id
        geofence_idx = row.gf_id
        last_point_idx = df_route.index[-1]
        gf_rad = df_geofences.loc[geofence_idx, "rad"]  # radio de la geocerca
        list_rad_gf.append(gf_rad)
        list_gps_gf.append(
            df_geofences.loc[geofence_idx, ["lat", "lon"]].values)
        if route_idx == 0:  # geocerca sobre punto de ruta "0" o entre "0" y "1"
            geofence_coord = df_geofences.loc[geofence_idx, [
                "x_axis", "y_axis"]].values
            route_center_coord = df_route.loc[route_idx, [
                "x_axis", "y_axis"]].values
            p_neighbour_coord = df_route.loc[route_idx +
                                             1, ["x_axis", "y_axis"]].values
            dist_p, nearest_p = pnt2line(
                geofence_coord, route_center_coord, p_neighbour_coord)
            list_order.append([route_idx, route_idx+1])
            list_dist.append(dist_p)
            gps_coord_c = df_route.loc[route_idx, ["lat", "lon"]].values
            gps_coord_p = df_route.loc[route_idx+1, ["lat", "lon"]].values
            list_gps_r.append([gps_coord_c, gps_coord_p])
            continue
        elif route_idx == last_point_idx:  # geocerca sobre el último punto de ruta o entre el último y antepenúltimo
            geofence_coord = df_geofences.loc[geofence_idx, [
                "x_axis", "y_axis"]].values
            a_neighbour_coord = df_route.loc[route_idx -
                                             1, ["x_axis", "y_axis"]].values
            route_center_coord = df_route.loc[route_idx, [
                "x_axis", "y_axis"]].values
            dist_a, nearest_a = pnt2line(
                geofence_coord, a_neighbour_coord, route_center_coord)
            list_order.append([route_idx-1, route_idx])
            list_dist.append(dist_a)
            gps_coord_a = df_route.loc[route_idx-1, ["lat", "lon"]].values
            gps_coord_c = df_route.loc[route_idx, ["lat", "lon"]].values
            list_gps_r.append([gps_coord_a, gps_coord_c])
            continue

        # Coordenadas de puntos de ruta y geocerca
        geofence_coord = df_geofences.loc[geofence_idx, [
            "x_axis", "y_axis"]].values
        a_neighbour_coord = df_route.loc[route_idx -
                                         1, ["x_axis", "y_axis"]].values
        route_center_coord = df_route.loc[route_idx, [
            "x_axis", "y_axis"]].values
        p_neighbour_coord = df_route.loc[route_idx +
                                         1, ["x_axis", "y_axis"]].values
        # Distancias entre geocerca y líneas vecinas
        dist_a, nearest_a = pnt2line(
            geofence_coord, a_neighbour_coord, route_center_coord)
        dist_p, nearest_p = pnt2line(
            geofence_coord, route_center_coord, p_neighbour_coord)

        if dist_a < dist_p:
            # print("\tGF {} entre puntos {} y {}".format(geofence_idx,route_idx-1,route_idx))
            list_order.append([route_idx-1, route_idx])
            list_dist.append(dist_a)
            gps_coord_a = df_route.loc[route_idx-1, ["lat", "lon"]].values
            gps_coord_c = df_route.loc[route_idx, ["lat", "lon"]].values
            list_gps_r.append([gps_coord_a, gps_coord_c])
        else:
            # print("\tGF {} entre puntos {} y {}".format(geofence_idx,route_idx,route_idx+1))
            list_order.append([route_idx, route_idx+1])
            list_dist.append(dist_p)
            gps_coord_c = df_route.loc[route_idx, ["lat", "lon"]].values
            gps_coord_p = df_route.loc[route_idx+1, ["lat", "lon"]].values
            list_gps_r.append([gps_coord_c, gps_coord_p])
    return {"order": list_order, "dist": list_dist, "gf_rad": list_rad_gf, "coord_gps_r": list_gps_r, "coord_gps_gf": list_gps_gf}

def get_gps_coord(coord_gps_r, coord_gps_gf):
    """
    Se determinan las coordenadas GPS de las geocercas SOBRE la trayectoria de
    la ruta.

    Este cálculo supone la latitud y longitud cómo variables lineales, aplicando
    la función "pnt2line" entre las coordenadas de la geocerca y la recta correspondiente
    entre los puntos de ruta por dónde la geocerca iría ubicada.

    Parameters
    ----------
    coord_gps_r: list[list,list]
        Lista con las coordenadas de los puntos de recta de la ruta
    coord_gps_gf: list
        Lista con las coordenadas de los puntos de la geocerca

    Returns
    -------
    np.array
        Coordenadas con las coordenadas de los puntos de la geocerca SOBRE la ruta
    """
    # Calculo de coordenadas GPS de geocerca SOBRE LA RUTA
    a_gps = coord_gps_r[0]
    p_gps = coord_gps_r[1]
    dist, gf_gps = pnt2line(coord_gps_gf, a_gps, p_gps)
    return gf_gps

def merge_gf(min_dist_df,df_route,df_geofences):
    full_route_df = pd.DataFrame(columns=['lat', 'lon','id'])
    lim_inf = 0
    lim_sup = min_dist_df.loc[0, "order"][0]
    for idx, row in min_dist_df.iterrows():
        segment_route_df = df_route.loc[lim_inf:lim_sup, ['lat', 'lon','id']]
        full_route_df = pd.concat([full_route_df, segment_route_df])
        geofence = pd.DataFrame({'lat':[row.coord_gps_gf[0]], 'lon':[row.coord_gps_gf[1]], 'dist':[row.dist],'gf_rad':[row.gf_rad],'id':[df_geofences.loc[row.gf_id,"id"]]})
        full_route_df = pd.concat([full_route_df, geofence])
        lim_inf = row.order[1]
        try:
            lim_sup = min_dist_df.loc[idx+1, "order"][0]
        except:  # al llegar a la última fila
            lim_sup = df_route.index[-1]  # Último registro de puntos de ruta
            segment_route_df = df_route.loc[lim_inf:lim_sup, ['lat', 'lon','id']]
            full_route_df = pd.concat([full_route_df, segment_route_df])
    full_route_df.insert(4, "gf_valid", full_route_df.gf_rad >= full_route_df.dist)
    full_route_df = full_route_df.reset_index().drop(columns=["index"])
    return full_route_df

### Objetivo

Se tiene:

    - df de coordenadas gps de ruta
    - df de coordenadas gps de geocercas
        - tiene además un radio de geocerca

Se requiere:

    - df de coordenadas gps en dónde las coordenadas de ruta y geocercas se encuentren organizados de manera lógica

    Condiciones:

        - Ubicar las geocercas entre puntos de ruta en un orden lógico que no altere el paso por todos los puntos de ruta
        - Una geocerca es válida para ser agregada a la ruta, si la ruta y el círculo dibujado por el radio de la geocerca se intersectan

### Solución:

In [89]:
# Todos los procesos se desarrollan sobre un plano cartesiano generado a partir 
# de las coordenadas gps disponibles. Se toma cómo origen el punto de partida 
# de la ruta del recorrido

# ENTRADAS:
df_route = pd.read_csv("route2.csv") # Dataframe de puntos del recorrido
df_geofences = pd.read_csv("gf_route_2.csv") # Dataframe de puntos de geocerca

# DESARROLLO:
df_route['id'] = df_route.index
df_geofences['id'] = df_geofences.index
origin = df_route.loc[0, ["lat", "lon"]] # Origen del recorrido
df_route = get_axis(df_route, origin) # Coordenadas x,y de la ruta
df_geofences = get_axis(df_geofences, origin) # Coordenadas x,y de las geocercas
print("Ruta:")
display(df_route)
print("Geocercas:")
display(df_geofences)
dist_df = euclidean_dist(df_route, df_geofences)
print("Distancia entre puntos:")
display(dist_df)
min_dist_df = dist_df.idxmin(axis=0).reset_index().rename(columns={"index": "gf_id", 0: "r_id"}) # identificación de distancia mínima
# gf_id: id de geocerca
# r_id: id de punto de ruta
values_within = pd.DataFrame(gf_within_r(min_dist_df, df_route, df_geofences)) # geocercas dentro de ruta
min_dist_df = pd.merge(min_dist_df, values_within, left_index=True, right_index=True)
min_dist_df = min_dist_df.sort_values(
    by="order").reset_index().drop(columns="index") # puntos de ruta en orden ascendente
min_dist_df["coord_gps_gf"] = min_dist_df.apply(
    lambda x: get_gps_coord(x["coord_gps_r"], x["coord_gps_gf"]), axis=1) # se ajustan la columna "coord_gps_gf"
print("Orden:")
display(min_dist_df)
full_route_df = merge_gf(min_dist_df,df_route,df_geofences)
print("Resultado:")
display(full_route_df)

Ruta:


,lat,lon,id,x_axis,y_axis
0,4.746628,-74.091267,0,0.000000,0.000000
1,4.746211,-74.090275,1,109.927061,-46.368284
2,4.745671,-74.089695,2,174.198932,-106.413545
3,4.745542,-74.089191,3,230.048971,-120.757690
4,4.744062,-74.089550,4,190.266899,-285.326182
5,4.742025,-74.090071,5,132.533029,-511.830247
6,4.741630,-74.090320,6,104.940450,-555.752243
7,4.738160,-74.086206,7,560.827476,-941.598639
8,4.736856,-74.082507,8,970.726870,-1086.596823
9,4.732738,-74.076824,9,1600.480387,-1544.497531


Geocercas:


,lat,lon,note,rad,id,x_axis,y_axis
0,4.712394,-74.071542,bulevar_niza,20,0,2185.797661,-3806.647119
1,4.683359,-74.045340,migracion_colombia,30,1,5089.334813,-7035.191814
2,4.736713,-74.082689,cementerio_suba,10,2,950.558801,-1102.497698
3,4.688444,-74.064570,iserra_100,40,3,2958.389869,-6469.765612


Distancia entre puntos:


,0,1,2,3
0,4389.564182,8683.043976,1455.700246,7114.066185
1,4295.222423,8581.267732,1349.841124,7026.647394
2,4211.681131,8495.088524,1262.900743,6945.787839
3,4172.617143,8451.157207,1217.763465,6910.408501
4,4047.449094,8340.356855,1116.159962,6775.676781
5,3882.230527,8192.931713,1008.986719,6594.123193
6,3859.829559,8174.798120,1006.976159,6566.409042
7,3293.786711,7592.052192,421.638485,6025.689629
8,2979.105678,7235.241098,25.682462,5738.406609
9,2336.646543,6505.369237,785.978321,5109.029637


Orden:


,gf_id,r_id,order,dist,gf_rad,coord_gps_r,coord_gps_gf
0,2,8,"[7, 8]",21.716464,10,"[[4.73816, -74.086206], [4.736856, -74.082507]]","[4.7368972608777575, -74.08262404293468]"
1,0,14,"[13, 14]",100.967780,20,"[[4.719085, -74.073991], [4.708008, -74.071483]]","[4.71219282831019, -74.07243050829665]"
2,3,17,"[17, 18]",68.937503,40,"[[4.689323, -74.065456], [4.685673, -74.051815]]","[4.689043038712659, -74.06440971180257]"
3,1,20,"[19, 20]",8.170983,30,"[[4.684665, -74.047972], [4.682507, -74.043335]]","[4.683425662939752, -74.04530897592754]"


Resultado:


,lat,lon,id,dist,gf_valid,gf_rad
0,4.746628,-74.091267,0,NaN,False,NaN
1,4.746211,-74.090275,1,NaN,False,NaN
2,4.745671,-74.089695,2,NaN,False,NaN
3,4.745542,-74.089191,3,NaN,False,NaN
4,4.744062,-74.089550,4,NaN,False,NaN
5,4.742025,-74.090071,5,NaN,False,NaN
6,4.741630,-74.090320,6,NaN,False,NaN
7,4.738160,-74.086206,7,NaN,False,NaN
8,4.736897,-74.082624,2,21.716464,False,10.0
9,4.736856,-74.082507,8,NaN,False,NaN


In [88]:
import plotly.express as px
full_route_df["is_gf"] = full_route_df["dist"].notna()
fig = px.line(full_route_df, x="lon", y="lat", markers=True, color="is_gf")
# fig = px.scatter(full_route_df[full_route_df.is_gf == True], x="lon", y="lat", color="is_gf")
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.show()

In [74]:
df_route.loc[lim_inf:lim_sup, ['lat', 'lon']]

,lat,lon
20,4.682507,-74.043335
21,4.681518,-74.041567
22,4.682777,-74.043082
23,4.683586,-74.042757


In [79]:
df_route.reset_index().rename(columns={"index":"id"})

,id,lat,lon,x_axis,y_axis
0,0,4.746628,-74.091267,0.000000,0.000000
1,1,4.746211,-74.090275,109.927061,-46.368284
2,2,4.745671,-74.089695,174.198932,-106.413545
3,3,4.745542,-74.089191,230.048971,-120.757690
4,4,4.744062,-74.089550,190.266899,-285.326182
5,5,4.742025,-74.090071,132.533029,-511.830247
6,6,4.741630,-74.090320,104.940450,-555.752243
7,7,4.738160,-74.086206,560.827476,-941.598639
8,8,4.736856,-74.082507,970.726870,-1086.596823
9,9,4.732738,-74.076824,1600.480387,-1544.497531


In [83]:
df_route

,lat,lon,id,x_axis,y_axis
0,4.746628,-74.091267,0,0.000000,0.000000
1,4.746211,-74.090275,1,109.927061,-46.368284
2,4.745671,-74.089695,2,174.198932,-106.413545
3,4.745542,-74.089191,3,230.048971,-120.757690
4,4.744062,-74.089550,4,190.266899,-285.326182
5,4.742025,-74.090071,5,132.533029,-511.830247
6,4.741630,-74.090320,6,104.940450,-555.752243
7,4.738160,-74.086206,7,560.827476,-941.598639
8,4.736856,-74.082507,8,970.726870,-1086.596823
9,4.732738,-74.076824,9,1600.480387,-1544.497531
